In [1]:
%run SAT_help_functions.ipynb

## OUM

In [2]:
data_dir = "/Users/wangjiashun/Desktop/Reserach on SAT/datasets/Evans2021/Rae2014/dataBySubject/sub"
sample_size = 34
SAT_conds = ["Speed", "Accuracy"]

In [3]:
class BoundSpeedAcc(Bound):
    name = "BoundSpeedAcc"
    required_parameters = ["Bspeed", "Bacc"]
    required_conditions = ['Cond']
    def get_bound(self, conditions, *args, **kwargs):
        assert self.Bacc > 0
        assert self.Bspeed > 0
        if conditions['Cond'] == 'Speed':
            return self.Bspeed
        elif conditions['Cond'] == 'Accuracy':
            return self.Bacc
        else:
            print("Non-implemented SAT conditions")

In [4]:

class DriftConstantSpeedAcc(Drift):
    name = "DriftConstantSpeedAcc"
    required_parameters = ["Vspeed","Vacc"]
    required_conditions = ['Cond']
        
    def get_drift(self, x, t, conditions, **kwargs):
        if conditions['Cond'] == 'Speed':
            return self.Vspeed
        elif conditions['Cond'] == 'Accuracy':
            return self.Vacc
        else: 
            print("Non-implemented SAT conditions")

In [5]:

class NondecisionSpeedAcc(OverlayNonDecision):
    name = "NondecisionSpeedAcc"
    required_parameters = ["tSpeed", 'tAcc']
    required_conditions = ['Cond']
        
    def get_nondecision_time(self, conditions, **kwargs):
        if conditions['Cond'] == 'Speed':
            return self.tSpeed 
        elif conditions['Cond'] == 'Accuracy':
            return self.tAcc
        else: 
            print("Non-implemented SAT conditions")

In [8]:
def get_DDM_result_sub_2v2a2ter(samp, v = [0,6], a = [0.5,8], ter=[0.1,1], T_dur=5):
    
    model_fit = Model(name='Simple model (fitted)',
                      drift=DriftConstantSpeedAcc(Vspeed = Fittable(minval=v[0], maxval=v[1]),
                                                  Vacc = Fittable(minval=v[0], maxval=v[1])), 
                      noise=NoiseConstant(noise=1),
                      bound=BoundSpeedAcc(
                          Bspeed=Fittable(minval=a[0], maxval=a[1]),
                          Bacc=Fittable(minval=a[0], maxval=a[1])),
                      
                      overlay=NondecisionSpeedAcc(
                          tSpeed=Fittable(minval=ter[0], maxval=ter[1]),
                          tAcc=Fittable(minval=ter[0], maxval=ter[1]),

                      ),
                      dx=.001, dt=.001, T_dur=T_dur)
    #set_N_cpus(4)
    fit_adjust_model(samp, model_fit,
                     fitting_method="differential_evolution",
                     lossfunction=LossRobustBIC, verbose=False)
    #print(model_fit.get_model_parameters())
    df = pd.DataFrame(data = np.array(model_fit.get_model_parameters()).reshape(1,6),
                      columns = model_fit.get_model_parameter_names())
    
    #df['Participant']=i+1
    #df['SAT'] = SAT
    df['BIC'] = model_fit.get_fit_result().value()
    df = df.rename(columns={'Vspeed': 'Drift Rate Speed',
                            'Vacc': 'Drift Rate Accuracy',
                            'Bspeed': 'Boundary Separation Speed',
                            'Bacc': 'Boundary Separation Accuracy',
                            'tSpeed': 'Non-decision Time Speed',
                            'tAcc': 'Non-decision Time Accuracy'})
    return df

In [15]:
df_all = pd.DataFrame()
for i in range(sample_size):
    print(i)
    path = os.path.join(data_dir+str(i+1)+".csv")
    df = pd.read_csv(path, sep = ",")
    #df = df[df["cond"]==SAT] # A condition
    df = df[df["Time"]<5]
    print("Accuracy: "+str(df["Correct"].mean()))
    print(df["Time"].mean())
    #samp = Sample.from_pandas_dataframe(df, rt_column_name="Time", correct_column_name="Correct")
    #df = get_DDM_result_sub_2v2a2ter(samp, a = [0.2,8])# since many a is at 0.5 so we lower the bound
    
    #df_all = pd.concat((df_all,df))
    #print(df)
df_OUM = df_all.copy()


0
Accuracy: 0.8532974427994616
0.5219838492597577
1
Accuracy: 0.7867679928475637
0.4622109968708091
2
Accuracy: 0.5936030103480715
0.4100729068673566
3
Accuracy: 0.8838248436103664
0.4414142091152815
4
Accuracy: 0.6502369668246446
0.4101526066350711
5
Accuracy: 0.8616071428571429
0.4194035714285714
6
Accuracy: 0.8079499776686021
0.448263957123716
7
Accuracy: 0.8434115523465704
0.47139124548736466
8
Accuracy: 0.8467741935483871
0.47722222222222227
9
Accuracy: 0.7833482542524619
0.4367148612354521
10
Accuracy: 0.5611577964519141
0.3834262371615313
11
Accuracy: 0.7707581227436823
0.5233596570397111
12
Accuracy: 0.6863553113553114
0.45062408424908423
13
Accuracy: 0.5769593956562795
0.3642360717658168
14
Accuracy: 0.7886251679355127
0.5364514106583071
15
Accuracy: 0.6010879419764279
0.43666636446056206
16
Accuracy: 0.6644052464947987
0.3639846223428313
17
Accuracy: 0.711996114618747
0.4558411850412822
18
Accuracy: 0.925
0.6228812499999999
19
Accuracy: 0.6385271665918276
0.40986708576560393


In [7]:
df_OUM.to_csv("DDM2v2a2terresultRae2014.csv")

In [10]:
df_OUM

,Drift Rate Speed,Drift Rate Accuracy,Boundary Separation Speed,Boundary Separation Accuracy,Non-decision Time Speed,Non-decision Time Accuracy,BIC
0,1.503307,1.895208,0.578727,0.669732,0.229096,0.296671,-442.773732
0,1.107020,1.925903,0.502216,0.544400,0.229942,0.295430,-1208.863644
0,0.322676,0.461770,0.485210,0.554946,0.163437,0.164342,472.495645
0,2.402430,2.655798,0.480646,0.506259,0.283592,0.300069,-3412.463930
0,0.513983,0.765940,0.485685,0.562142,0.165734,0.167133,161.540172
0,2.047592,2.417996,0.511282,0.505880,0.238215,0.270048,-1404.683287
0,1.616551,1.983600,0.445294,0.478007,0.273557,0.308781,-2238.099768
0,1.821804,1.825880,0.424777,0.659537,0.257039,0.265792,-1621.098900
0,1.712400,2.159261,0.512074,0.573220,0.276961,0.289490,-964.920746
0,1.286365,1.751664,0.459197,0.514402,0.239854,0.271719,-1358.090190
